In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 250
num_epochs = 200
learning_rate = 1e-5
samples = 30000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (28, 28), color = 'white')
        #img = np.reshape(img,(28,28,1))
        #print(np.shape(img))
        row,col,ch= np.shape(img)
        mean = random.randint(1,6)
        var = random.random()
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([0,0,5,5],fill='red',outline='red')
        #im2_pos1.show()
        #print(np.shape(np.transpose(np.asarray(im2_pos1),(2,0,1))))
        im2_pos1 = np.reshape(im2_pos1,(28,28,3))
        #print(np.shape(im2_pos1))
        data1.append(np.array(np.transpose(np.asarray(im2_pos1),(2,0,1)),dtype=np.float32))

        img_1 = Image.new('RGB', (28, 28), color = 'white')
        row_1,col_1,ch_1= np.shape(img_1)
        mean_1 = random.randint(1,6)
        var_1 = random.random()
        sigma_1 = var_1**0.5
        gauss_pos2 = np.random.normal(mean_1,sigma_1,(row_1,col_1,ch_1))
        noisy_pos2 = img_1 + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw_1 = ImageDraw.Draw(im2_pos2)
        draw_1.rectangle([0,0,5,5],fill='red',outline='red')
        im2_pos2 = np.reshape(im2_pos2,(28,28,3))
        data2.append(np.array(np.transpose(np.asarray(im2_pos2),(2,0,1)),dtype=np.float32))
    elif switch==1:
        img_blank = Image.new('RGB', (28, 28), color = 'white')
        row_blank,col_blank,ch_blank= np.shape(img_blank)
        mean_blank = random.randint(1,6)
        var_blank = random.random()
        sigma_blank = var_blank**0.5
        gauss_blank_pos1 = np.random.normal(mean_blank,sigma_blank,(row_blank,col_blank,ch_blank))
        noisy_blank_pos1 = img_blank + gauss_blank_pos1
        im2_blank_pos1 = Image.fromarray(noisy_blank_pos1,'RGB')
        im2_blank_pos1 = np.reshape(im2_blank_pos1,(28,28,3))
        data1.append(np.array(np.transpose(np.asarray(im2_blank_pos1),(2,0,1)),dtype=np.float32))

        img_blank_1 = Image.new('RGB', (28, 28), color = 'white')
        row_blank_1,col_blank_1,ch_blank_1= np.shape(img_blank_1)
        mean_blank_1 = random.randint(1,6)
        var_blank_1 = random.random()
        sigma_blank_1 = var_blank_1**0.5
        gauss_blank_pos2 = np.random.normal(mean_blank_1,sigma_blank_1,(row_blank_1,col_blank_1,ch_blank_1))
        noisy_blank_pos2 = img_blank_1 + gauss_blank_pos2
        im2_blank_pos2 = Image.fromarray(noisy_blank_pos2,'RGB')
        im2_blank_pos2 = np.reshape(im2_blank_pos2,(28,28,3))
        data2.append(np.array(np.transpose(np.asarray(im2_blank_pos2),(2,0,1)),dtype=np.float32))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (28,28), color = 'white')
row,col,ch= np.shape(img)
mean = 0
var = 1
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
im2 = np.reshape(im2,(28,28,3))
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class ConvNet(torch.nn.Module):
 
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        
        # 28x28x3 => 28x28x8
        self.conv_1 = torch.nn.Conv2d(in_channels=3,
                                      out_channels=8,
                                      kernel_size=(5, 5),
                                      stride=(1, 1),
                                      padding=1) # (1(28-1) - 28 + 3) / 2 = 1
        # 28x28x8 => 14x14x8
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(14-1) - 28 + 2) = 0                                       
        # 14x14x8 => 14x14x16
        self.conv_2 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=16,
                                      kernel_size=(5,5),
                                      stride=(1, 1),
                                      padding=1) # (1(14-1) - 14 + 3) / 2 = 1                 
        # 14x14x16 => 7x7x16                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(7-1) - 14 + 2) = 0


        
    def forward(self, x):
        out = self.conv_1(x)
        #out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        #out = F.relu(out)
        out = self.pool_2(out)

        return out

    
model = ConvNet(num_classes=1)


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,weight_decay=1e-4)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul((torch.transpose(fx,3,2)), fx) / Nsamples

    covg = torch.matmul((torch.transpose(gy,3,2)), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))
    print("b:", model(b.reshape(1,3,28,28)))


ModuleNotFoundError: No module named 'cv2'

In [182]:
data3=[]

for i in range(5):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = random.randint(1,6)
    var = random.random()
    sigma = var**0.5
    gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
    noisy_pos1 = img + gauss_pos1
    im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
    draw = ImageDraw.Draw(im2_pos1)
    draw.rectangle([0,0,5,5],fill='red',outline='red')
    im2_pos1 = np.reshape(im2_pos1,(28,28,3))

    data3.append(np.asarray(im2_pos1))

    y3 = np.array(data3, dtype=np.float32)
    #y1 = np.array(data2, dtype=np.float32)
    #y3 = y3[:,:,:,0]
    #y1 = y1[:,:,:,0]
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].reshape(1,3,28,28)))


x:  tensor([[[[ 9.9066e-01, -9.6283e-02, -2.3827e-01,  1.7512e-02, -2.2789e-02],
          [ 6.1857e-01, -1.8400e+00, -2.5946e+00, -2.9432e+00, -2.0657e+00],
          [ 6.7600e-01, -1.9240e+00, -2.2486e+00, -2.2040e+00, -2.7188e+00],
          [ 5.2706e-01, -1.9797e+00, -2.8590e+00, -2.7216e+00, -2.1971e+00],
          [ 1.7860e+00, -2.5758e+00, -3.0075e+00, -2.3288e+00, -2.6380e+00]],

         [[ 1.4385e+00,  9.4360e-01,  1.7733e+00,  1.1223e+00,  1.3098e+00],
          [ 2.9656e-01, -2.5479e+00, -3.0177e+00, -2.7908e+00, -2.6089e+00],
          [ 2.5557e-01, -3.2420e+00, -2.6832e+00, -3.3756e+00, -2.9805e+00],
          [ 2.3174e-02, -3.1487e+00, -3.4308e+00, -3.3576e+00, -3.0294e+00],
          [-6.5590e-03, -3.0051e+00, -3.1095e+00, -2.7769e+00, -3.1237e+00]],

         [[ 9.9761e-01, -1.3092e-01, -2.6259e-01,  2.5968e-03, -5.5112e-02],
          [ 6.1836e-01, -1.8359e+00, -2.5950e+00, -2.9556e+00, -2.0380e+00],
          [ 6.8891e-01, -1.9228e+00, -2.2248e+00, -2.2046e+00, -2.69